# Resume-Chat Feature With Title-Generation

## Scope

This document defines an implementation-ready design for a multi-thread “Resume Chat” capability using Streamlit + LangGraph, including the homework extension for human-friendly chat titles.
All outputs are deterministic, self-contained, auditable, and production-grade.

---

# 1. Architecture

## 1.1 Components

* **Frontend:** Streamlit
* **LLM Orchestration:** LangGraph
* **Checkpointer:** MemorySaver
* **Persistence (Optional Homework DB):** Not enabled; titles stored in SessionState

## 1.2 Data Flow (Declarative)

User Input → Streamlit UI → LangGraph Stream → Checkpointer (thread-scoped) → Streamlit UI

## 1.3 Trust Boundaries

Browser ↔ Streamlit (TLS 1.3+) ↔ LangGraph Runtime (Local Privileged Execution)
No cross-tenant data exposure; session isolation only in memory.

## 1.4 Mermaid Diagram

```mermaid
flowchart TD
    UI[Streamlit UI] -->|user input| LG[LangGraph Agent]
    LG -->|stream| UI
    LG -->|read/write| CP[MemorySaver Checkpointer]
    UI -->|select thread| CP
```

---

# 2. Thread Management Implementation

## 2.1 Session Initialization

```python
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langgraph_tool_backend import chatbot
import uuid

if 'message_history' not in st.session_state:
    st.session_state['message_history'] = []

if 'chat_threads' not in st.session_state:
    st.session_state['chat_threads'] = {}

if 'thread_id' not in st.session_state:
    st.session_state['thread_id'] = str(uuid.uuid4())
```

Data model:

```text
chat_threads: Dict[str, Dict]
{
  thread_id: {
      "title": str,
      "last_message": str
  }
}
```

---

## 2.2 Thread Creation Logic

```python
def generate_thread_id() -> str:
    return str(uuid.uuid4())

def new_thread():
    tid = generate_thread_id()
    st.session_state['thread_id'] = tid
    st.session_state['message_history'] = []
    st.session_state['chat_threads'][tid] = {
        "title": "New Conversation",
        "last_message": ""
    }
```

---

## 2.3 Sidebar Selection

```python
st.sidebar.title("Chat Sessions")

if st.sidebar.button("New Chat"):
    new_thread()

for tid, meta in reversed(st.session_state['chat_threads'].items()):
    label = meta['title']
    if st.sidebar.button(label, key=tid):
        resume_thread(tid)
```

---

## 2.4 Resume Logic

```python
def resume_thread(tid: str):
    st.session_state['thread_id'] = tid
    state = chatbot.get_state({"configurable": {"thread_id": tid}})
    messages = state.values.get('messages', [])
    buffer = []
    for m in messages:
        role = 'user' if isinstance(m, HumanMessage) else 'assistant'
        buffer.append({"role": role, "content": m.content})
    st.session_state['message_history'] = buffer
```

---

# 3. Streaming Send Path

```python
for m in st.session_state['message_history']:
    with st.chat_message(m['role']):
        st.write(m['content'])

user_input = st.chat_input("Message")

if user_input:
    tid = st.session_state['thread_id']
    st.session_state['message_history'].append({"role": "user", "content": user_input})
    st.session_state['chat_threads'][tid]['last_message'] = user_input

    CONFIG = {"configurable": {"thread_id": tid}}

    with st.chat_message("assistant"):
        def stream_out():
            for chunk, meta in chatbot.stream(
                {"messages": [HumanMessage(content=user_input)]},
                config=CONFIG,
                stream_mode="messages"
            ):
                if isinstance(chunk, AIMessage):
                    yield chunk.content
        assistant_msg = st.write_stream(stream_out())

    st.session_state['message_history'].append({"role": "assistant", "content": assistant_msg})
```

---

# 4. Homework: Auto-Generated Conversation Titles

## 4.1 Requirement

Titles must:

* be deterministic
* be generated from the first user message
* update only once
* avoid manual naming
* resemble ChatGPT behavior

## 4.2 Deterministic Title Extraction Strategy

Algorithm:

1. consume first user message
2. extract top-k nouns (fallback: first 3 words)
3. title-case output

This avoids LLM title hallucination and works offline.

## 4.3 Implementation

```python
import re
import string

def extract_title(text: str) -> str:
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text[:80]
    words = [w.strip(string.punctuation) for w in text.split()]
    title = " ".join(words[:6]).title()
    return title or "Conversation"
```

Integration hook:

```python
if user_input:
    tid = st.session_state['thread_id']
    meta = st.session_state['chat_threads'][tid]
    if meta['title'] == "New Conversation":
        st.session_state['chat_threads'][tid]['title'] = extract_title(user_input)
```

Resulting UX example:

```
User first message: "Explain q-learning with code"
Title stored: "Explain Q Learning With Code"
```

---

# 5. Q&A (Technical Clarity)

**Q:** Why generate titles on first user message instead of after N messages?
**A:** Guarantees deterministic naming and immediate sidebar UX without requiring streaming buffering.

**Q:** Why not use LLM to generate titles?
**A:** Non-deterministic outputs complicate caching and reproducibility; homework variant avoids extra inference cost.

**Q:** Can titles be edited by user?
**A:** Yes. Add a text_input bound to `chat_threads[tid]['title']`.

**Q:** How to persist threads across restarts?
**A:** Replace MemorySaver with SQLite/PostgreSQL durable storage and maintain thread_id foreign key.

---




---



```python
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langgraph_tool_backend import chatbot
import uuid
import re
import string


# ---------------------- UTILITIES ----------------------

def generate_thread_id():
    return str(uuid.uuid4())

def extract_title(text: str) -> str:
    # clean + shorten + make title case
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    words = [w.strip(string.punctuation) for w in text.split()]
    title = " ".join(words[:6]).title()
    return title if title else "Conversation"

def create_new_thread():
    tid = generate_thread_id()
    st.session_state.thread_id = tid
    st.session_state.message_history = []
    st.session_state.chat_threads[tid] = {"title": "New Conversation"}
    return tid

def load_thread_from_backend(thread_id):
    state = chatbot.get_state({"configurable": {"thread_id": thread_id}})
    msgs = state.values.get("messages", [])
    converted = []
    for m in msgs:
        role = "user" if isinstance(m, HumanMessage) else "assistant"
        converted.append({"role": role, "content": m.content})
    return converted


# ---------------------- SESSION INIT ----------------------

if "chat_threads" not in st.session_state:
    st.session_state.chat_threads = {}

if "thread_id" not in st.session_state:
    create_new_thread()

if "message_history" not in st.session_state:
    st.session_state.message_history = []


# ---------------------- SIDEBAR UI ----------------------

st.sidebar.title("Chat Sessions")

if st.sidebar.button("New Chat"):
    create_new_thread()

st.sidebar.subheader("My Conversations")

for tid, meta in reversed(st.session_state.chat_threads.items()):
    title = meta["title"]
    if st.sidebar.button(title, key=tid):
        st.session_state.thread_id = tid
        st.session_state.message_history = load_thread_from_backend(tid)


# ---------------------- MAIN CHAT UI ----------------------

for msg in st.session_state.message_history:
    with st.chat_message(msg["role"]):
        st.write(msg["content"])

user_input = st.chat_input("Message")

if user_input:
    tid = st.session_state.thread_id

    # set title on first message
    if st.session_state.chat_threads[tid]["title"] == "New Conversation":
        st.session_state.chat_threads[tid]["title"] = extract_title(user_input)

    # show user's message
    st.session_state.message_history.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.write(user_input)

    CONFIG = {"configurable": {"thread_id": tid}}

    # stream assistant response
    with st.chat_message("assistant"):
        def stream_output():
            for chunk, _ in chatbot.stream(
                {"messages": [HumanMessage(content=user_input)]},
                config=CONFIG,
                stream_mode="messages"
            ):
                if isinstance(chunk, AIMessage):
                    yield chunk.content

        assistant_reply = st.write_stream(stream_output())

    st.session_state.message_history.append({"role": "assistant", "content": assistant_reply})
```

---

# 💡 **Why This Version Is “Easy to Understand”**

Because it organizes the app in the most intuitive way:

### **1. Utilities at top**

small, clear, single-purpose functions

### **2. Session initialization**

ensures app never breaks on first load

### **3. Sidebar**

handles navigation + new chat creation

### **4. Main panel**

handles messaging + streaming output

### **5. No surprises**

backend loads history only when user selects a thread

---

# 🧠 **Quick Q&A (for clarity)**

**Q: Where are chat messages stored?**
→ In `MemorySaver` inside LangGraph, keyed by `thread_id`.

**Q: Why not store messages in Streamlit only?**
→ Because then “Resume Chat” would reset on refresh — backend storage prevents that.

**Q: When is title generated?**
→ On the **first user message**, just like ChatGPT.

**Q: Can user rename threads manually?**
→ Yes, trivial to add later with `st.text_input`.

**Q: Can these be persisted across app restarts?**
→ Not with MemorySaver; requires SQLite/Postgres (next step).

---

---

